In [6]:
import psycopg2
import random
from datetime import datetime, timedelta
from faker import Faker

# Criar uma instância de Faker
fake = Faker()

# Função para conectar ao banco de dados
def conectar():
    try:
        conn = psycopg2.connect(
            dbname="ecommerce",
            user="postgres",
            password="20669884",
            host="localhost",
            port="5432"
        )
        print("Conexão com o banco de dados foi bem-sucedida.")
        return conn
    except Exception as e:
        print(f"Falha ao conectar ao banco de dados: {e}")
        return None

# Função para inserir dados nas tabelas
def inserir_dados(conn):
    cur = conn.cursor()
    try:
        # Inserir registros na tabela Clientes
        id_clientes = []
        for _ in range(10000):
            nome = fake.name()[:99]  # Limitar a 100 caracteres
            endereco = fake.address()[:999]  # Limitar a 1000 caracteres
            telefone = fake.phone_number()[:19]  # Limitar a 20 caracteres
            email = fake.email()[:99]  # Limitar a 100 caracteres
            data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=90)
            cur.execute("""
                INSERT INTO Clientes (Nome, Endereco, Telefone, Email, Data_Nascimento)
                VALUES (%s, %s, %s, %s, %s)
                RETURNING ID_Cliente
            """, (nome, endereco, telefone, email, data_nascimento))
            id_cliente = cur.fetchone()[0]
            id_clientes.append(id_cliente)

        # Inserir registros na tabela Produtos
        id_produtos = []
        for _ in range(10000):
            nome_produto = fake.word().capitalize()[:49]  # Limitar a 50 caracteres
            descricao = fake.text(max_nb_chars=200)[:199]  # Limitar a 200 caracteres
            preco = round(random.uniform(1.0, 1000.0), 2)
            quantidade_estoque = random.randint(0, 1000)
            cur.execute("""
                INSERT INTO Produtos (Nome, Descricao, Preco, Quantidade_Estoque)
                VALUES (%s, %s, %s, %s)
                RETURNING ID_Produto
            """, (nome_produto, descricao, preco, quantidade_estoque))
            id_produto = cur.fetchone()[0]
            id_produtos.append(id_produto)

        # Inserir registros na tabela Pedidos usando IDs válidos de Clientes
        for _ in range(10000):
            data_pedido = fake.date_time_between(start_date='-2y', end_date='now')
            status = random.choice(['Pendente', 'Processando', 'Concluído', 'Cancelado'])
            valor_total = round(random.uniform(20.0, 5000.0), 2)
            id_cliente = random.choice(id_clientes)  # Selecionar um ID_Cliente existente
            cur.execute("""
                INSERT INTO Pedidos (Data_Pedido, Status, Valor_Total, ID_Cliente)
                VALUES (%s, %s, %s, %s)
                RETURNING ID_Pedido
            """, (data_pedido, status, valor_total, id_cliente))
            id_pedido = cur.fetchone()[0]

            # Inserir registros na tabela Itens_Pedido usando IDs válidos de Produtos
            for _ in range(random.randint(1, 5)):  # Inserir de 1 a 5 itens por pedido
                quantidade = random.randint(1, 20)
                preco_unitario = round(random.uniform(5.0, 200.0), 2)
                subtotal = round(quantidade * preco_unitario, 2)
                id_produto = random.choice(id_produtos)  # Selecionar um ID_Produto existente
                cur.execute("""
                    INSERT INTO Itens_Pedido (Quantidade, Preco_Unitario, Subtotal, ID_Pedido, ID_Produto)
                    VALUES (%s, %s, %s, %s, %s)
                """, (quantidade, preco_unitario, subtotal, id_pedido, id_produto))

                # Atualizar a quantidade de estoque do produto
                cur.execute("""
                    UPDATE Produtos
                    SET Quantidade_Estoque = Quantidade_Estoque - %s
                    WHERE ID_Produto = %s
                """, (quantidade, id_produto))

                # Inserir registros na tabela Pagamentos para cada item de pedido
                data_pagamento = data_pedido + timedelta(days=random.randint(1, 30))
                valor_pago = subtotal
                metodo_pagamento = random.choice(['Cartão de Crédito', 'Boleto', 'Pix', 'PayPal'])
                cur.execute("""
                    INSERT INTO Pagamentos (Data_Pagamento, Valor_Pago, Metodo_Pagamento, ID_Pedido)
                    VALUES (%s, %s, %s, %s)
                """, (data_pagamento, valor_pago, metodo_pagamento, id_pedido))

        # Commit das alterações
        conn.commit()
        print("Inserção de registros concluída com sucesso!")

    except Exception as e:
        print(f"Falha ao inserir registros: {e}")
        conn.rollback()

    finally:
        # Fechar o cursor e a conexão
        cur.close()
        conn.close()
        print("Conexão encerrada.")

# Função principal para executar o código
def main():
    conn = conectar()
    if conn:
        inserir_dados(conn)

# Executar o programa principal
if __name__ == "__main__":
    main()


Conexão com o banco de dados foi bem-sucedida.
Inserção de registros concluída com sucesso!
Conexão encerrada.
